# ZIP Auto Loader with Nested Extraction and Delta Manifest

This notebook:

- Uses **Auto Loader** to ingest ZIP files from a Unity Catalog Volume.
- Recursively extracts ZIP contents (including **nested ZIPs**) into an **extracted Volume**.
- Writes a **Delta manifest table** listing all extracted files, their origin ZIP, and paths.


In [0]:
import json
from clinical_data_standards_framework.utils import (
    create_zip_autoloader_stream,
    create_zip_batch_processor
)


{'source_root_dbfs': 'dbfs:/Volumes/aira/bronze_md/input_data/historical_data/zip',
 'target_root_dbfs': 'dbfs:/Volumes/aira/bronze_md/input_data/historical_data/extracted',
 'checkpoint_path': '/Volumes/aira/bronze_md/input_data/jobs/_checkpoints/zip_processor',
 'manifest_table': 'aira.bronze_md.trial_data_history'}

In [ ]:
# Get configuration from setup task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
services_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="services"))
pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")


In [ ]:
# Extract needed values from globals
catalog = globals_dict['catalog']
schema = globals_dict['bronze_schema']

# Get ZIP processor specific configuration from pipeline_config
extraction_config = pipeline_config['archives']['clinical_trial_data']['extraction']

source_volume = extraction_config['source_volume']
target_volume = extraction_config['target_volume']
source_subdir = extraction_config['source_subdir']
target_subdir = extraction_config['target_subdir']
manifest_table = extraction_config['manifest_table']
manifest_table_description = extraction_config.get('manifest_table_description', '')
status_table = extraction_config['status_table']
status_table_description = extraction_config.get('status_table_description', '')
checkpoint_subdir = extraction_config['checkpoint_subdir']
supported_extensions = extraction_config['supported_extensions']

# Get document categorization config (from extraction block)
document_categories = pipeline_config['archives']['clinical_trial_data']['extraction'].get('document_categories', {})

# Get streaming config for completion tracking
streaming_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="streaming_config", default="{}"))
completion_tracking = streaming_dict.get('completion_tracking', {})
required_tags_for_versioning = completion_tracking.get('required_tags_for_versioning', ['tsDTA'])

print(f"Catalog: {catalog}")
print(f"Schema: {schema}")
print(f"Source: /Volumes/{catalog}/{schema}/{source_volume}/{source_subdir}")
print(f"Target: /Volumes/{catalog}/{schema}/{target_volume}/{target_subdir}")
print(f"Manifest Table: {catalog}.{schema}.{manifest_table}")
print(f"Status Table: {catalog}.{schema}.{status_table}")
print(f"Checkpoint: /Volumes/{catalog}/{schema}/{source_volume}/{checkpoint_subdir}")
print(f"Supported Extensions: {supported_extensions}")
print(f"Document Categories Configured: {len(document_categories)} top-level categories")
print(f"Required Tags for Versioning: {required_tags_for_versioning}")


In [ ]:
# Construct paths using config values
source_root = f"/Volumes/{catalog}/{schema}/{source_volume}/{source_subdir}"
target_root = f"/Volumes/{catalog}/{schema}/{target_volume}/{target_subdir}"
manifest_table_full_name = f"{catalog}.{schema}.{manifest_table}"
status_table_full_name = f"{catalog}.{schema}.{status_table}"
checkpoint_path = f"/Volumes/{catalog}/{schema}/{source_volume}/{checkpoint_subdir}"

print(f"Source root: {source_root}")
print(f"Target root: {target_root}")
print(f"Manifest table: {manifest_table_full_name}")
print(f"Status table: {status_table_full_name}")
print(f"Checkpoint: {checkpoint_path}")


In [0]:
# Create Auto Loader streaming DataFrame for ZIP files
zip_stream_df = create_zip_autoloader_stream(spark, source_root)

# Display schema
zip_stream_df.printSchema()


root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)



In [0]:
# Create the batch processor using the utility function
# This function encapsulates all the ZIP extraction logic
# Only files with supported extensions will be extracted
# Documents will be categorized and tagged based on folder structure
# Status tracking enabled for document processing lifecycle

process_zip_batch = create_zip_batch_processor(
    spark=spark,
    target_root_path=target_root,
    manifest_table_full_name=manifest_table_full_name,
    status_table_full_name=status_table_full_name,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id,
    supported_extensions=supported_extensions,
    categories_config=document_categories,
    manifest_table_description=manifest_table_description,
    status_table_description=status_table_description,
    required_tags_for_versioning=required_tags_for_versioning
)

print("ZIP batch processor created successfully")
print(f"Will process files with extensions: {supported_extensions}")
print(f"Status tracking enabled - writing to: {status_table_full_name}")
if document_categories:
    print(f"Document categorization enabled with {len(document_categories)} top-level categories")
else:
    print("Document categorization not configured")


In [0]:
# Start the Auto Loader streaming query
# For one-time historical processing, use availableNow=True
# For continuous ingestion, change trigger to processingTime or remove trigger entirely

query = (
    zip_stream_df.writeStream
                 .foreachBatch(process_zip_batch)
                 .option('checkpointLocation', checkpoint_path)
                 .trigger(availableNow=True)  # change for continuous streaming if desired
                 .start()
)

query.awaitTermination()


In [0]:
# Cell 8: Conditional OPTIMIZE - trial_data_history table
# Only run OPTIMIZE when conditions are met to save compute resources

from datetime import datetime

# Get table stats
try:
    table_stats = spark.sql(f"DESCRIBE DETAIL {manifest_table_full_name}").first()
    num_files = table_stats.numFiles
    total_size_gb = table_stats.sizeInBytes / (1024**3)
    avg_file_size_mb = (total_size_gb * 1024) / num_files if num_files > 0 else 0
    
    print(f"\n{'='*80}")
    print(f"TABLE STATS: {manifest_table_full_name}")
    print(f"{'='*80}")
    print(f"Total files: {num_files}")
    print(f"Total size: {total_size_gb:.2f} GB")
    print(f"Avg file size: {avg_file_size_mb:.2f} MB")
    
    # Determine if OPTIMIZE is needed
    should_optimize = False
    reason = ""
    
    # We don't know exact doc count processed, so use file-based heuristics
    if num_files > 200 and avg_file_size_mb < 100:  # Condition 1: Small file problem
        should_optimize = True
        reason = f"Small file problem ({num_files} files, avg {avg_file_size_mb:.2f}MB)"
    elif num_files > 500:  # Condition 2: Too many files
        should_optimize = True
        reason = f"Too many files ({num_files})"
    elif total_size_gb > 1 and num_files > 100 and avg_file_size_mb < 200:  # Condition 3: Growing dataset with small files
        should_optimize = True
        reason = f"Dataset growing with sub-optimal file sizes ({total_size_gb:.2f}GB, {num_files} files)"
    
    if should_optimize:
        print(f"\n🔧 OPTIMIZE TRIGGERED: {reason}")
        print(f"{'='*80}")
        
        start_time = datetime.now()
        
        # Run OPTIMIZE with Z-ORDER
        spark.sql(f"""
            OPTIMIZE {manifest_table_full_name}
            ZORDER BY (current_status, document_tags)
        """)
        
        # Get stats after optimization
        table_stats_after = spark.sql(f"DESCRIBE DETAIL {manifest_table_full_name}").first()
        num_files_after = table_stats_after.numFiles
        
        duration = (datetime.now() - start_time).total_seconds()
        
        print(f"✓ OPTIMIZE completed in {duration:.1f} seconds")
        print(f"  Files: {num_files} → {num_files_after} ({num_files - num_files_after} merged)")
        print(f"{'='*80}")
    else:
        print(f"\n⏭️  OPTIMIZE SKIPPED: Conditions not met")
        print(f"  - Files: {num_files} (threshold: >200 with small files OR >500 total)")
        print(f"  - Avg file size: {avg_file_size_mb:.2f}MB (threshold: <100-200MB)")
        print(f"  - Total size: {total_size_gb:.2f}GB")
        print(f"{'='*80}")
        
except Exception as e:
    print(f"\n⚠️  Could not check OPTIMIZE conditions: {str(e)}")
    print("Continuing without optimization...")